# 나이 그룹 별 화재 건수

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Pretendard Variable'
plt.rcParams['axes.unicode_minus'] = False

# 경고 무시

fire_data = pd.read_csv('./results/gu_build_fire.csv')
age_data = pd.read_csv('./filtered_data/group_population_byGu.csv', encoding='utf-8')

In [19]:
fire_data = fire_data[['SGG_NM', '화재건수']].copy()

fire_data.rename(columns={'SGG_NM': '행정구역', '화재건수': '화재_건수'}, inplace=True)
fire_data.to_csv('./filtered_data/gu_fire_counts.csv', index=False, encoding='utf-8')

In [23]:
import matplotlib.pyplot as plt
import os
from scipy import stats

# 데이터 준비
# age_data 읽기 (FirePerAge.ipynb에서 생성한 파일)
age_ratio_data = pd.read_csv('./filtered_data/group_population_ratio_byGu.csv', encoding='utf-8')

# fire_data와 age_ratio_data 병합
merged_data = fire_data.merge(age_ratio_data, left_on='행정구역', right_on='자치구', how='inner')

print("병합된 데이터:")
print(merged_data.head())

# results 디렉터리 확인/생성
os.makedirs('./results', exist_ok=True)

# 나이대별 그래프 생성 및 저장
age_groups = ['미성년자_비율', '청년_비율', '중년_비율', '장년_비율', '고령_비율']

for age_group in age_groups:
    plt.figure(figsize=(10, 6))
    
    # 산점도 생성
    plt.scatter(merged_data[age_group], merged_data['화재_건수'], alpha=0.6, s=100)
    
    # 상관계수 및 p-value 계산
    correlation, p_value = stats.pearsonr(merged_data[age_group], merged_data['화재_건수'])
    
    # 회귀선 추가
    z = np.polyfit(merged_data[age_group], merged_data['화재_건수'], 1)
    p = np.poly1d(z)
    plt.plot(merged_data[age_group].sort_values(), p(merged_data[age_group].sort_values()), 
             "r--", alpha=0.8, linewidth=2, label=f'Regression line')
    
    # 제목 및 라벨
    plt.title(f'{age_group} 인구비율 vs 화재 건수\n(상관계수: {correlation:.3f}, p-value: {p_value:.4f})', 
              fontsize=14, fontweight='bold')
    plt.xlabel(f'{age_group} 인구 비율', fontsize=12)
    plt.ylabel('화재 건수', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # 각 점에 구명 라벨 추가
    for idx, row in merged_data.iterrows():
        plt.annotate(row['행정구역'], (row[age_group], row['화재_건수']), 
                    fontsize=8, alpha=0.7, xytext=(5, 5), textcoords='offset points')
    
    # 그래프 저장
    plt.tight_layout()
    plt.savefig(f'./results/{age_group}_vs_화재.png', dpi=300, bbox_inches='tight')
    print(f"✓ {age_group} 그래프 저장 완료: ./results/{age_group}_vs_화재.png")
    plt.close()

# 전체 상관계수 및 p-value 요약
print("\n=== 나이대별 화재 상관계수 및 p-value ===")
correlation_summary = {}
pvalue_summary = {}
for age_group in age_groups:
    correlation, p_value = stats.pearsonr(merged_data[age_group], merged_data['화재_건수'])
    correlation_summary[age_group] = correlation
    pvalue_summary[age_group] = p_value
    print(f"{age_group}: 상관계수 = {correlation:.4f}, p-value = {p_value:.4f}")

# 상관계수 및 p-value를 CSV로 저장
result_df = pd.DataFrame({
    'age_group': age_groups,
    'correlation': [correlation_summary[ag] for ag in age_groups],
    'p_value': [pvalue_summary[ag] for ag in age_groups]
})
result_df.to_csv('./results/age_fire_correlation_pvalue.csv', index=False, encoding='utf-8')
print("\n✓ 상관계수 및 p-value 요약 저장: ./results/age_fire_correlation_pvalue.csv")

병합된 데이터:
  행정구역  화재_건수  자치구  미성년자_비율  청년_비율  중년_비율  장년_비율  고령_비율
0  강남구    422  강남구    19.93  29.12  33.69   8.29   8.98
1  강동구    171  강동구    17.63  26.53  32.28  11.82  11.73
2  강북구    200  강북구    13.80  27.58  31.81  12.61  14.21
3  강서구    274  강서구    15.99  31.69  30.41  10.66  11.25
4  관악구    281  관악구    10.99  41.10  28.14   9.59  10.18
✓ 미성년자_비율 그래프 저장 완료: ./results/미성년자_비율_vs_화재.png
✓ 미성년자_비율 그래프 저장 완료: ./results/미성년자_비율_vs_화재.png
✓ 청년_비율 그래프 저장 완료: ./results/청년_비율_vs_화재.png
✓ 청년_비율 그래프 저장 완료: ./results/청년_비율_vs_화재.png
✓ 중년_비율 그래프 저장 완료: ./results/중년_비율_vs_화재.png
✓ 중년_비율 그래프 저장 완료: ./results/중년_비율_vs_화재.png
✓ 장년_비율 그래프 저장 완료: ./results/장년_비율_vs_화재.png
✓ 장년_비율 그래프 저장 완료: ./results/장년_비율_vs_화재.png
✓ 고령_비율 그래프 저장 완료: ./results/고령_비율_vs_화재.png

=== 나이대별 화재 상관계수 및 p-value ===
미성년자_비율: 상관계수 = 0.2878, p-value = 0.1629
청년_비율: 상관계수 = 0.0760, p-value = 0.7179
중년_비율: 상관계수 = 0.1423, p-value = 0.4974
장년_비율: 상관계수 = -0.5024, p-value = 0.0105
고령_비율: 상관계수 = -0.4782, p-value = 0.0156

✓ 상관계수 및 p

In [13]:
import matplotlib as mpl
mpl.get_cachedir()

'C:\\Users\\User\\.matplotlib'

In [17]:
mpl.matplotlib_fname()

'c:\\ProgramData\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\matplotlibrc'